In [1]:
import numpy as np
from scipy.interpolate import interp1d
import pyslha
import sys,imp
sys.path.append('../smodels')
from smodels.tools.physicsUnits import fb,pb,GeV,TeV,mm,m

In [4]:
#Get CMS efficiencies:
M1effFile = '../smodels-database/13TeV/CMS/CMS-PAS-EXO-16-036-eff/orig/effmap_M1_stau_cons_mrec300_trim.txt'
M2effFile = '../smodels-database/13TeV/CMS/CMS-PAS-EXO-16-036-eff/orig/effmap_M2_stau_cons_mrec300_trim.txt'
M1effData = np.loadtxt(M1effFile)
M2effData = np.loadtxt(M2effFile)
M1effFunc = interp1d(M1effData[:,0],M1effData[:,1],bounds_error=False,fill_value=0.)
M2effFunc = interp1d(M2effData[:,0],M2effData[:,1],bounds_error=False,fill_value=0.)

In [5]:
#Get SLHA data
slhaFile = '../data/IDM/2sigma_smallDetaM_xsecs/20000000291_paramcard.slha'
slha = pyslha.readSLHAFile(slhaFile)
totxsec = sum([max([x.value for x in proc.get_xsecs(sqrts=13000)]) for proc in slha.xsections.values()])*pb
mass = slha.blocks['MASS'][37]*GeV
width = slha.decays[37].totalwidth*GeV
ctau = (1.967e-16)*GeV*m/width
print('mass=',mass,'\ntotxsec=',totxsec,'\nwidth=',width,'\nctau=',ctau)

mass= 5.22E+02 [GeV] 
totxsec= 1.38E-03 [pb] 
width= 8.48E-19 [GeV] 
ctau= 2.32E+02 [m]


In [6]:
#Compute the relevant Flong and Fprompt factors:
l_inner=1.*mm
gb_inner=1.3
l_outer=10.*m
gb_outer=1.3
Flong = np.exp(-(l_outer/(gb_outer*ctau)).asNumber())
Fprompt = 1. - np.exp(-(l_inner/(gb_inner*ctau)).asNumber())
print("Flong=",Flong,"Fprompt=",Fprompt)

Flong= 0.9673752295900478 Fprompt= 3.3168769050240243e-06


In [7]:
#Get efficincies for the relevant models/mass:
eff1 = M1effFunc(mass.asNumber(GeV))
eff2 = M2effFunc(mass.asNumber(GeV))
print('eff1=',eff1,'eff2=',eff2)

eff1= 0.4392181731317246 eff2= 0.2752386056288934


In [8]:
#Compute A^0 BR to HSCP and MET (to be compressed into M1/M2 models)
BRmet = sum([x.br for x in slha.decays[36].decays if 35 in x.ids])
BRhscp = sum([x.br for x in slha.decays[36].decays if 37 in x.ids])
print('BRmet=',BRmet,'BRhscp=', BRhscp)

BRmet= 0.48424024 BRhscp= 0.5157597599999999


In [14]:
#Compute the xsecs for the M1 and M2 models (assuming Fprompt ~ 0)
M1xsec = 0.*fb
M2xsec = 0.*fb
for proc in slha.xsections.values():
    xsecval = proc.get_xsecs(sqrts=13000)[0].value*pb
    proc.pidsfinal = sorted(proc.pidsfinal)
    if proc.pidsfinal == [-37,37]:
        M1xsec += xsecval*Flong**2
    if proc.pidsfinal in [[-37,35],[35,37]]:
        M2xsec += xsecval*Flong
    if proc.pidsfinal in [[-37,36],[36,37]]:
        M1xsec += xsecval*Flong**2*BRhscp
        M2xsec += xsecval*Flong*BRmet
    if proc.pidsfinal in [[35,36]]:
        M2xsec += xsecval*Flong*BRhscp        
    if proc.pidsfinal in [[36,36]]:
        M1xsec += xsecval*Flong**2*BRhscp**2
        M2xsec += 2*xsecval*Flong*BRhscp*BRmet
print('M1xsec=',M1xsec.asNumber(fb),'M2xsec=',M2xsec.asNumber(fb))

M1xsec= 0.4567677719602473 M2xsec= 0.7512163770388193


In [13]:
#Compute effective cross-section, including efficiencies (should match theory prediction for EMs)
print('M1xsecEff=',(M1xsec*eff1).asNumber(fb),'M2xsecEff=',(M2xsec*eff2).asNumber(fb))
print('xsecEff=',(M1xsec*eff1+M2xsec*eff2).asNumber(fb))

M1xsecEff= 0.20062070634582801 M2xsecEff= 0.20676374814175366
xsecEff= 0.4073844544875817


In [11]:
#Load SModelS result for 13 TeV:
f = slhaFile+'.py'
smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
for exp in smodelsDict['ExptRes']:
    if exp['AnalysisSqrts (TeV)'] != 13.:
        continue
    print(exp['AnalysisID'])
    print('theory prediction=',exp['theory prediction (fb)'])
    for tx in exp['TxNames weights (fb)']:
        print('\t tx weight=',tx,exp['TxNames weights (fb)'][tx])

CMS-PAS-EXO-16-036
theory prediction= 0.40743791017187686
	 tx weight= THSCPM1b 0.20065835103619814
	 tx weight= THSCPM2b 0.20677955913567872
CMS-PAS-EXO-16-036
theory prediction= 0.45686406221243264
	 tx weight= THSCPM1b 0.45686406221243264
